### Importing libraries

In [1]:
from barmnist import BarMNIST
from torchvision import transforms
import torch
from cdm.utils import visualize_image
import os
from diffusers import DDPMScheduler
import logging
from dotenv import load_dotenv
from cdm.scm import EdgeType, Vertex, Edge, SCM
from cdm.constants import IMAGE_CHANNELS, IMAGE_RESOLUTION
from cdm.causal_image_model import CausalImageModel
from cdm.utils import mnist_y_labels
import time

# Setup the logger
load_dotenv()
logger = logging.getLogger(__name__)
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler("mnist.log"),  # Log to file
        logging.StreamHandler()          # Log to console
    ],
    datefmt="%Y-%m-%d %H:%M:%S"
)
logger.setLevel(os.getenv('LOG_LEVEL'))
logger.info(os.getenv('LOG_LEVEL'))

/home/asardesai/Projects/Causal-Diffusion-Model/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-16 16:33:41 - INFO - DEBUG


In [2]:
BATCH_SIZE = int(os.getenv('BATCH_SIZE'))
NUM_EPOCHS = int(os.getenv('NUM_EPOCHS'))
MODEL_PATH = 'mnist_model.pt'
DEVICE_NAME = os.getenv('DEVICE')
DEVICE = torch.device(DEVICE_NAME) if torch.cuda.is_available() else 'cpu'
LEARNING_RATE = 1e-4

### Load the Dataset

In [3]:
cg = 'frontdoor'

BarMNIST(cg=cg, ow=True)
trans_f = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                                ])
train_set = BarMNIST(cg=cg, root='./', env='train', transform=trans_f)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True,
                                            drop_last=True)

Preparing BAR MNIST
Causal Diagram:frontdoor
BAR MNISTfrontdoor dataset already exists


In [4]:
x_sample = None
x_sample_batch = None
for idx, (x, y) in enumerate(train_loader):
    x_sample_batch = x
    x_sample = x_sample_batch[0]
    print(mnist_y_labels(y))
    break

visualize_image(x_sample)

[tensor([1, 7, 2, 9, 0, 4, 6, 9]), tensor([1, 1, 0, 1, 0, 0, 1, 0]), tensor([1, 0, 0, 0, 1, 1, 1, 0])]


In [5]:
noise_scheduler = DDPMScheduler(num_train_timesteps=1000)
img2 = x_sample.unsqueeze(0)
noise = torch.randn_like(img2)
noisy_image = noise_scheduler.add_noise(img2, noise, torch.tensor([80]))
noisy_image = noisy_image[0]
visualize_image(noisy_image)

### Initialize the Model

The Diffusion model will represent the node I. Node I takes the generative features as inputs along with the noised version of the image.
It will then attempt to predict the noise aka denoising the image to genearte the original input image through a series of timesteps.

In [6]:
d_vertex = Vertex('Digit', (10,))
b_vertex = Vertex('Bar', (2,))
c_vertex = Vertex('Color', (2,))
edge_list = [
    Edge(d_vertex, c_vertex, EdgeType.DIRECTED),
    Edge(c_vertex, b_vertex, EdgeType.DIRECTED),
    Edge(d_vertex, b_vertex, EdgeType.BI_DIRECTED)
]
# This will be useful when we pass the input into the generator and for the loss on the labels.
vertex_order = [d_vertex, b_vertex, c_vertex]
scm = SCM(edge_list).to(DEVICE)
# values = scm(x_sample_batch.to(DEVICE))
if os.path.exists(MODEL_PATH):
    cim = torch.load(MODEL_PATH, weights_only=False, map_location=DEVICE)
    logger.info("Loading model from checkpoint...")
else:
    cim = CausalImageModel(scm)
    logger.info("Initializing a new model...")
cim.to(DEVICE)

2025-05-16 16:33:58 - INFO - Loading model from checkpoint...


CausalImageModel(
  (scm): SCM(
    (layers): ModuleDict(
      (Vertex(label:'Color, dimensions: '(2,)'')): SCMFunction(
        (cnn1): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (cnn2): Conv2d(32, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (cnn3): Conv2d(16, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (linear): Linear(in_features=49, out_features=2, bias=True)
        (pooling): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (relu): ReLU()
      )
      (Vertex(label:'Bar, dimensions: '(2,)'')): SCMFunction(
        (cnn1): Conv2d(8, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (cnn2): Conv2d(32, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (cnn3): Conv2d(16, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (linear): Linear(in_features=49, out_features=2, bias=True)
        (pooling): MaxPool2d(kernel_size=2, stride=2, padding=0, d

### Train the Model

In [ ]:
# Train the model
cim.train()

num_params = sum(p.numel() for p in cim.parameters())
logger.info(f"Number of parameters: {num_params / 1e6:.2f} million")

# Our loss function
loss_fn1 = torch.nn.MSELoss()
loss_fn2 = torch.nn.CrossEntropyLoss()
loss_fn3 = torch.nn.NLLLoss()
# The optimizer
opt = torch.optim.Adam(cim.parameters(), lr=LEARNING_RATE)

# Keeping a record of the losses for later viewing
losses = []

# The training loop
for epoch in range(NUM_EPOCHS):
    if epoch % 10 == 0:
        torch.save(cim, MODEL_PATH)
        logger.info('Saved the model checkpoint...')
    start_time = time.time()
    for idx, (x, y) in enumerate(train_loader):
        start_time2 = time.time()
        # Get some data and prepare the corrupted version
        x = x.to(DEVICE)  # Data on the GPU
        y_labels = mnist_y_labels(y.to(DEVICE))
        noise_amount = torch.randn_like(x).to(DEVICE)  # Pick random noise amounts
        timestep = torch.randint(0, 1000, (x.shape[0],)).to(DEVICE) # Pick a random timestep to train on
        noisy_x = noise_scheduler.add_noise(x, noise_amount, timestep) # Create our noisy x
        # Get the model prediction
        pred_noise, gen_factors, ae_factors = cim(x, noisy_x, timestep, vertex_order)  # <<< Using timestep  always, adding .sample
        # Calculate the losses for both scm generative factors and the predicted noise.
        loss1 = loss_fn1(pred_noise, noise_amount)
        loss2 = 0
        loss3 = 0
        loss4 = 0
        # We do not directly apply loss on high dimensional space. However, it is worth testing how this works maybe.
        # for logits, labels, in zip(gen_factors, y_labels):
        #     loss2 += loss_fn2(logits, labels)
        for v_high, (v_low, v_high_constructed), labels, in zip(gen_factors, ae_factors, y_labels):
            # Reconstruction loss between v_high and v_high_constructed. This is MSE.
            loss2 += loss_fn1(v_high_constructed, v_high)
            # NLLoss between the v_low and true labels. We take log and add a small epsilon so log doesnt become zero.
            loss3 += loss_fn3(torch.log(v_low + 1e-9), labels)
            # Trying to keep the high dimensional variables closer to our labels
            loss4 += loss_fn2(v_high, labels)
            
        # Normalize the loss
        loss2 /= len(y_labels)
        loss3 /= len(y_labels)
        loss4 /= len(y_labels)
        total_loss = loss1 + loss2 + loss3 + loss4
        
        # Backprop and update the params:
        opt.zero_grad()
        total_loss.backward()
        opt.step()
        # Store the loss for later
        losses.append(total_loss.item())
        logger.debug(f"Time it would take for an epoch is {(time.time() - start_time2) * 30000 / 60 / BATCH_SIZE} minutes with batch size: {BATCH_SIZE}")
    end_time = time.time()
    # Print our the average of the loss values for this epoch:
    avg_loss = sum(losses[-len(train_loader) :]) / len(train_loader)
    logger.info(f"Finished epoch {epoch}. Average loss for this epoch: {avg_loss:05f}. Time taken to run epoch: {int((end_time-start_time)/60)} minutes.")


torch.save(cim, MODEL_PATH)

2025-05-16 16:33:59 - INFO - Number of parameters: 107.03 million
2025-05-16 16:33:59 - INFO - Saved the model checkpoint...
2025-05-16 16:34:00 - DEBUG - Time it would take for an epoch is 56.56750500202179 minutes with batch size: 8
2025-05-16 16:34:00 - DEBUG - Time it would take for an epoch is 12.069374322891235 minutes with batch size: 8
2025-05-16 16:34:00 - DEBUG - Time it would take for an epoch is 11.363878846168518 minutes with batch size: 8
2025-05-16 16:34:01 - DEBUG - Time it would take for an epoch is 11.540815234184265 minutes with batch size: 8
2025-05-16 16:34:01 - DEBUG - Time it would take for an epoch is 11.643245816230774 minutes with batch size: 8
2025-05-16 16:34:01 - DEBUG - Time it would take for an epoch is 11.525169014930725 minutes with batch size: 8
2025-05-16 16:34:01 - DEBUG - Time it would take for an epoch is 11.617109179496765 minutes with batch size: 8
2025-05-16 16:34:01 - DEBUG - Time it would take for an epoch is 11.397525668144226 minutes with ba

KeyboardInterrupt: 

### Evaluation

Try to perform inference by generating an image using the Causal SCM model.


In [ ]:
cim.eval()
# Create random noise
latents = torch.rand((BATCH_SIZE, IMAGE_CHANNELS, IMAGE_RESOLUTION, IMAGE_RESOLUTION)).to(DEVICE) * noise_scheduler.init_noise_sigma
# The original image we set to random during generation
# I = torch.rand(8, IMAGE_CHANNELS, IMAGE_RESOLUTION, IMAGE_RESOLUTION).to(DEVICE)
I = x_sample_batch.to(DEVICE)

# Sampling loop
noise_scheduler.set_timesteps(1000)  # Adjust steps as needed
reset_scm = True
for t in noise_scheduler.timesteps:
    with torch.no_grad():
        noise_pred, gen_factors, ae_factors = cim(I, latents, t, vertex_order, reset_scm=reset_scm)
        reset_scm = False # After the first generation, we reuse the scm.
    latents = noise_scheduler.step(noise_pred, t, latents).prev_sample

# Decode from the latent space into the image space
pred_image = latents
image_list = [image for image in pred_image]
new_image = torch.cat(image_list, dim=1)
visualize_image(new_image)

### 

In [ ]:
print("Digit")
print(torch.argmax(ae_factors[0][0], dim=1))
print("Bar")
print(torch.argmax(ae_factors[1][0], dim=1))
print("Color")
print(torch.argmax(ae_factors[2][0], dim=1))

Digit
tensor([4, 9, 9, 8, 9, 6, 3, 7], device='cuda:0')
Bar
tensor([1, 1, 1, 1, 1, 0, 1, 1], device='cuda:0')
Color
tensor([0, 0, 0, 0, 0, 0, 1, 0], device='cuda:0')


In [ ]:
image_list = [image for image in x_sample_batch]
new_image = torch.cat(image_list, dim=1)
visualize_image(new_image)

### Performing an Intervention

Now let us try to perform an intervention on the SCM

In [ ]:
# Create random noise
latents = torch.rand((BATCH_SIZE, IMAGE_CHANNELS, IMAGE_RESOLUTION, IMAGE_RESOLUTION)).to(DEVICE) * noise_scheduler.init_noise_sigma
# Sampling loop
noise_scheduler.set_timesteps(1000)  # Adjust steps as needed

# Our intervention.
# intervened_d_value = torch.tensor([0,0,0,0,0,0,0,0,1,0], dtype=torch.float32).unsqueeze(0).expand(BATCH_SIZE, -1).to(DEVICE)
# intervention_dict = {d_vertex:intervened_d_value}

intervented_c_value = torch.tensor([1, 0], dtype=torch.float32).unsqueeze(0).expand(BATCH_SIZE, -1).to(DEVICE)
intervention_dict = {c_vertex: intervented_c_value}

for t in noise_scheduler.timesteps:
    with torch.no_grad():
        noise_pred, gen_factors, ae_factors = cim(I, latents, t, vertex_order, reset_scm=False, intervention_dict=intervention_dict)
        # No need to recompute the intervention the second time.
        intervention_dict=None
    latents = noise_scheduler.step(noise_pred, t, latents).prev_sample
# Decode from the latent space into the image space
pred_image = latents
image_list = [image for image in pred_image]
new_image = torch.cat(image_list, dim=1)
visualize_image(new_image)

In [ ]:
print("Digit")
print(torch.argmax(ae_factors[0][0], dim=1))
print("Bar")
print(torch.argmax(ae_factors[1][0], dim=1))
print("Color")
print(torch.argmax(ae_factors[2][0], dim=1))

Digit
tensor([4, 9, 9, 8, 9, 6, 3, 7], device='cuda:0')
Bar
tensor([1, 1, 1, 1, 1, 0, 1, 1], device='cuda:0')
Color
tensor([0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
